In [1]:
import sys
from pyspark.sql import SparkSession

def addTag(L): 
#输入是['一灯大师','黄蓉,0.16341463414634147|黄药师,0.036585365853658534|']的形式 注意最后的|
#输出是[['一灯大师','一灯大师'],[['黄蓉', '黄蓉', 0.16341463414634147],['黄药师', '黄药师', 0.036585365853658534']]]
    pn = L[0]
    rlist = L[1][:len(L[1])-1].split('|') #去除结尾的|，注意task3.out的格式是否被修改，需要相应修改
    for i in range(0, len(rlist)):
        rlist[i] = rlist[i].split(',')
        rlist[i].insert(1, rlist[i][0])
    pnt = [L[0], L[0]]
    return [pnt, rlist]

def createInputFile(inPath, spark):
    lines  = spark.read.text(inPath) \
        .rdd.map(lambda x:x[0]) \
        .map(lambda x:x.split('\t')) \
        .map(addTag)
    return lines

def findTags(L):
    #输入是[['一灯大师','一灯大师'],[['黄蓉', '黄蓉', 0.16341463414634147],['黄药师', '黄药师', 0.036585365853658534']]]
    #目的是统计所有相关人物的标签贡献值（权重和）,并以它作为人物的标签
    rlist = L[1]
    ctbt = {} #contribution
    for r in rlist:
        if r[1] not in ctbt:
            ctbt[r[1]] = 0
        ctbt[r[1]] += float(r[2])
    max_w, max_n = max(zip(ctbt.values(), ctbt.keys())) #找出最大权值及对应的人名
    return [L[0][0], max_n]
        
def updateRecords(record, tagMap):
    #输入是[[人物1,标签1], [[相关人物1,标签1,权重1], [相关人物2,标签2,权重2]]]
    #输出结构一样，要求把所有人物的标签更新
    if record[0][0] in tagMap:
        record[0][1] = tagMap[record[0][0]]
    rlist = record[1]
    for i in range(0, len(rlist)):
        if rlist[i][0] in tagMap:
            rlist[i][1] = tagMap[rlist[i][0]]
    return record
    
def drive():
    spark = SparkSession\
        .builder\
        .appName("Task5")\
        .getOrCreate()
    records = createInputFile('./../result/task3.out', spark)
    
    times = 21
    
    for i in range(0, times):
        #下面将每条记录map成一个[人物,标签]结构,并全部输出
        tags = records.map(findTags).collect()
        tagMap = {}
        for tag in tags:
            tagMap[tag[0]] = tag[1]
        #print(tagMap)
        #下面根据标签更新记录，最后一次迭代不需要再更新记录
        if i != times-1:
            records = records.map(lambda x: updateRecords(x, tagMap))
    fout = open('./../result/task5.out', 'w', encoding='utf-8')
    tags = records.map(findTags).sortBy(lambda x: x[1],ascending=True).collect()
    fout.write('Tag' + ',' + 'Name' + '\n')
    for tag in tags:
        fout.write(tag[1] + ',' + tag[0] + '\n')
    fout.close()
    
drive()